In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import json
import time
import copy
import linecache
import gzip
import pickle
import itertools
from sklearn.preprocessing import MinMaxScaler
import datetime
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import Dataset
from torch.nn import functional as F

import sys, argparse, os
import numpy as np
import random
import sys, argparse, os

import torch.nn.functional as F
import torch.utils.data as torchdata

import sys
sys.path.insert(0, '../')
sys.path.append("/scratch2/ml_flood/mlflood/")
from pathlib import Path
from mlflood.conf import PATH_DATA
from mlflood.conf import rain_const, waterdepth_diff_const
import h5py
import pandas as pd

from torch.utils.tensorboard import SummaryWriter
from mlflood.utils import new_log
from models.utae import UTAE
from models.CNNrolling import CNNrolling
from models.unet3d import UNet3D
from models.unet import UNet
from dataset import load_test_dataset, dataloader_args_test

from training import *
#from developing_suite4 import *

In [ ]:
path_exp = "/scratch2/ml_flood/data/checkpoints/709/cluster/unet_l1/experiment_0/"
path_exp = "/scratch2/ml_flood/data/checkpoints/709/unet_temp/experiment_2/"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using: ', device)
args = []   

In [ ]:
parser = argparse.ArgumentParser(description="evaluation")
parser.add_argument("--catchment_kwargs", default='./default_catchment_kwargs.yml', type=str, 
                    help="path to catchment kwargs saved in yml file")

exp_hp = ['--catchment_kwargs=../mlflood/exp_yml/exp_9_utae.yml']
args = parser.parse_args(exp_hp)

with open(args.catchment_kwargs) as file:
    catchment_kwargs = yaml.full_load(file)
    
# Always in eval mode
catchment_kwargs['fix_indexes'] = True 

In [ ]:
dataloaders = {}
dataset = load_test_dataset(catchment_kwargs)
                                                             
dataloaders["test"] = dataloader_args_test(dataset, catchment_num = catchment_kwargs['num'])
dataset_test = dataloaders["test"]

In [ ]:
model = UNet(args)  
file_path1 = path_exp + "model.pth.tar"
model.load_state_dict(torch.load(file_path1))
model.cuda()

In [ ]:
from evaluation import predict_event 

event_num = 0
start_ts = 5
# timestep = 12

pred_cnn, gt_cnn, mask_cnn = predict_event(model, dataset, event_num, 'unet', start_ts=5, ar = False, T = None)

In [ ]:
zoom = [500,1000,500,1000]
save_folder = "/scratch2/ml_flood/data/checkpoints/709/cluster/utae_L1/experiment_0/results/c_recons/"
plot_answer_sample(pred_cnn, gt_cnn, mask_cnn, ts=1, zoom=None, show_diff=False, global_scale=True, save_folder = None, model_name = 'cnn')

In [ ]:
zoom = [1500,2000,0,500]
plot_answer_sample(pred_cnn, gt_cnn, mask_cnn, ts=1, zoom=zoom, show_diff=False, global_scale=True, save_folder = None, model_name = 'cnn')